In [15]:
!pip install torchsummary
!pip install tqdm
!pip install torch
!pip install scikit-image
!pip install sklearn
!pip install cv2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [11]:
# importing the libraries
from torchsummary import summary
import pandas as pd
import numpy as np
from skimage.io import imread, imsave
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from skimage.transform import rotate
from skimage.util import random_noise
from skimage.filters import gaussian
from scipy import ndimage
import random

In [12]:
# # loading images
# train_img = []
# for img_name in tqdm(data['image_names']):
#     image_path = 'emergency_vs_non-emergency_dataset/images/' + img_name
#     img = imread(image_path)
#     img = img/255
#     train_img.append(img)

# train_x = np.array(train_img)
# train_y = data['emergency_or_not'].values
# train_x.shape, train_y.shape

In [13]:
import cv2
import os
import glob
img_dir = "face_images/*"
files = glob.glob(img_dir)
data = []

for f1 in files:
    img = imread(f1)
    img = img/255
    data.append(img)

train_x = np.array(data)
train_x.shape

ModuleNotFoundError: No module named 'cv2'

In [14]:
train_x, val_x= train_test_split(train_x, test_size = 0.1, random_state = 13)
(train_x.shape), (val_x.shape)

NameError: name 'train_x' is not defined

In [ ]:
train_x[1,:,:].shape

In [ ]:
from skimage.color import rgb2gray

In [3]:
#https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image
def cropND(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [4]:
# def rgb2gray(rgb):
#     return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

In [5]:
final_train_data = []
final_target_train = []

for i in tqdm(range(train_x.shape[0])):
    
    final_train_data.append(rgb2gray(train_x[i]))
                            
    final_train_data.append(random.uniform(0.6,1.01)*train_x[i,:,:])

    final_train_data.append(train_x[i])
    
    final_train_data.append(rotate(train_x[i], angle=45, mode = 'wrap'))
    
    final_train_data.append(np.fliplr(train_x[i]))
    
    final_train_data.append(np.flipud(train_x[i]))
    
#     final_train_data.append(random_noise(train_x[i],var=0.2**2))
    

NameError: name 'tqdm' is not defined

In [6]:
fig,ax = plt.subplots(nrows=1,ncols=10,figsize=(20,20))
for i in range(10):
   
    ax[i].imshow(final_train_data[i+30])
    #x[i].imshow(final_train_data[i+30])
    ax[i].axis('off')

NameError: name 'plt' is not defined